In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.python.keras.layers import Input, Dense
import os
for dirname, _, filenames in os.walk('/content/drive'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [5]:
from tensorflow.keras.layers import Bidirectional, Dropout, Activation, Dense, LSTM
from tensorflow.python.keras.layers import CuDNNLSTM
from tensorflow.keras.models import Sequential
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, SimpleRNN , GRU , Activation
from tensorflow.keras.layers import (BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense)
from keras import optimizers
import tensorflow as tf
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D , AveragePooling1D
from sklearn.preprocessing import (StandardScaler, OrdinalEncoder,LabelEncoder, MinMaxScaler, OneHotEncoder)
from sklearn.preprocessing import Normalizer, MaxAbsScaler , RobustScaler, PowerTransformer
import matplotlib.pyplot as plt
from keras.utils.generic_utils import get_custom_objects
from keras import backend as K
from keras.layers import Activation
from sklearn.preprocessing import LabelEncoder

def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))
  
def custom_activation(x, beta = 1):
        return (K.sigmoid(beta * x) * x)

get_custom_objects().update({'custom_activation': Activation(custom_activation)})

from IPython.display import display_html
from itertools import chain,cycle
def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

from IPython.display import display, HTML

css = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(css))

In [6]:
dataset = pd.read_csv('stokl60s.csv')
dataset.head()

,DNo,Mal grubu,Malzeme,Fiili miktar,Brim,Net,Miktar(Litre),Teslimat no,RfOmr,DeplSure,Stok yas,Raf omru k
0,3,7,16,144,1,10.0,1440.0,80313000,180,135,3,1.66
1,3,7,16,117,1,10.0,1170.0,80313000,180,135,3,1.66
2,3,7,16,495,1,10.0,4950.0,80313000,180,135,3,1.66
3,3,6,13,396,2,10.0,3960.0,80313002,120,75,11,8.94
4,3,7,41,132,2,10.0,1320.0,80313002,180,135,32,17.58


In [7]:
dataset = pd.read_csv('stokl60s.csv')
dataset.head()

,DNo,Mal grubu,Malzeme,Fiili miktar,Brim,Net,Miktar(Litre),Teslimat no,RfOmr,DeplSure,Stok yas,Raf omru k
0,3,7,16,144,1,10.0,1440.0,80313000,180,135,3,1.66
1,3,7,16,117,1,10.0,1170.0,80313000,180,135,3,1.66
2,3,7,16,495,1,10.0,4950.0,80313000,180,135,3,1.66
3,3,6,13,396,2,10.0,3960.0,80313002,120,75,11,8.94
4,3,7,41,132,2,10.0,1320.0,80313002,180,135,32,17.58


In [8]:
dataset=dataset.drop(['DNo', 'Raf omru k'],axis=1)
dataset.head()

,Mal grubu,Malzeme,Fiili miktar,Brim,Net,Miktar(Litre),Teslimat no,RfOmr,DeplSure,Stok yas
0,7,16,144,1,10.0,1440.0,80313000,180,135,3
1,7,16,117,1,10.0,1170.0,80313000,180,135,3
2,7,16,495,1,10.0,4950.0,80313000,180,135,3
3,6,13,396,2,10.0,3960.0,80313002,120,75,11
4,7,41,132,2,10.0,1320.0,80313002,180,135,32


In [9]:
#X = dataset.iloc[:, :5].values
y = dataset.iloc[:, -1].values

In [10]:
dataset=dataset.drop(['Stok yas'],axis=1)
X = dataset.iloc[:, :].values

In [11]:
sc = StandardScaler()
X = sc.fit_transform(dataset)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.33, random_state=0)

# LR

In [13]:
lr = LinearRegression()
lr.fit(x_train,y_train)

LinearRegression()

In [14]:
y_pred = lr.predict(x_test)

In [15]:
print('linear model coeff (Theta_0): {}'.format(lr.coef_))
print('linear model intercept (Theta_1): {:.3f}'.format(lr.intercept_))
print('R-squared score (training): {:.3f}'.format(lr.score(x_train, y_train)))
print('R-squared score (test): {:.3f}'.format(lr.score(x_test, y_test)))

linear model coeff (Theta_0): [  0.25927972   3.26573944 -10.37904935   2.36234186  -0.61209369
   7.93921912  -0.36683447  37.00031537 -33.16219475]
linear model intercept (Theta_1): 17.200
R-squared score (training): 0.212
R-squared score (test): 0.191


# Polynomial with Ridge

In [16]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures

In [17]:
poly = PolynomialFeatures(degree=3)


X_F1_poly_try = poly.fit_transform(X) #using the avlues in X_my, generate new set of features 
#called X_F1-poly

print("X_F1_poly_try", X_F1_poly_try)

X_F1_poly = poly.fit_transform(X) #using the values in X_F1, generate new set of features called X_F1


X_train1, X_test1, y_train1, y_test1 = train_test_split(X_F1_poly, y, random_state = 0)
linreg = LinearRegression().fit(X_train1, y_train1)

#print('(poly deg 2) linear model coeff (w):\n{}'.format(linreg.coef_))
#print('(poly deg 2) linear model intercept (b): {:.3f}'.format(linreg.intercept_))
print('(poly deg 3) R-squared score (training): {:.3f}'
     .format(linreg.score(X_train1, y_train1)))
print('(poly deg 3) R-squared score (test): {:.3f}\n'
     .format(linreg.score(X_test1, y_test1)))


X_train2, X_test2, y_train2, y_test2 = train_test_split(X_F1_poly, y,
                                                   random_state = 0)
linreg = Ridge(alpha=1).fit(X_train2, y_train2)

#print('(poly deg 2) linear model coeff (w):\n{}'.format(linreg.coef_))
#print('(poly deg 2) linear model intercept (b): {:.3f}'.format(linreg.intercept_))
print('(poly deg 3 + ridge) R-squared score (training): {:.3f}'
     .format(linreg.score(X_train2, y_train2)))
print('(poly deg 3 + ridge) R-squared score (test): {:.3f}'
     .format(linreg.score(X_test2, y_test2)))


X_F1_poly_try [[ 1.          0.38613706 -0.45575952 ... -0.01697772 -0.01492066
  -0.01311283]
 [ 1.          0.38613706 -0.45575952 ... -0.01697772 -0.01492066
  -0.01311283]
 [ 1.          0.38613706 -0.45575952 ... -0.01697772 -0.01492066
  -0.01311283]
 ...
 [ 1.         -0.89092843 -0.15667136 ...  9.04438027  8.76733931
   8.49878447]
 [ 1.          2.30173528  1.45840469 ... -0.01697772 -0.01492066
  -0.01311283]
 [ 1.          0.38613706 -0.57539478 ... -0.01697772 -0.01492066
  -0.01311283]]
(poly deg 3) R-squared score (training): 0.674
(poly deg 3) R-squared score (test): 0.548

(poly deg 3 + ridge) R-squared score (training): 0.635
(poly deg 3 + ridge) R-squared score (test): 0.570


In [18]:
y_pred = linreg.predict(X_test2)

In [19]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Assuming y_true and y_pred are arrays of actual and predicted values, respectively

# Calculate mean squared error (MSE)
mse = mean_squared_error(y_test2, y_pred)

# Calculate root mean squared error (RMSE)
rmse = np.sqrt(mse)

# Calculate R-squared (R2)
r2 = r2_score(y_test2, y_pred)

# Calculate mean absolute error (MAE)
mae = mean_absolute_error(y_test2, y_pred)

# Print the results
print("MSE:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)
print("MAE:", mae)

MSE: 101.55102415543986
RMSE: 10.077252807955146
R-squared: 0.5699233548934359
MAE: 6.642041565614428


# SVM

In [22]:
from sklearn.svm import LinearSVC

clf = LinearSVC().fit(x_train, y_train)
#print('intercept_', clf.intercept_)
#print('coef_', clf.coef_)

print('R-Squared Score of Linear SVC classifier on training set: {:.2f}'
     .format(clf.score(x_train, y_train)))
print('R-Squared Score of Linear SVC classifier on test set: {:.2f}'
     .format(clf.score(x_test, y_test)))

R-Squared Score of Linear SVC classifier on training set: 0.12
R-Squared Score of Linear SVC classifier on test set: 0.10


C:\Users\ozgur\anaconda3\envs\tf\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [23]:
y_pred = clf.predict(x_test)

In [24]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Assuming y_true and y_pred are arrays of actual and predicted values, respectively

# Calculate mean squared error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calculate root mean squared error (RMSE)
rmse = np.sqrt(mse)

# Calculate R-squared (R2)
r2 = r2_score(y_test, y_pred)

# Calculate mean absolute error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Print the results
print("MSE:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)
print("MAE:", mae)

MSE: 274.714709371293
RMSE: 16.574519883583143
R-squared: -0.14248094673659284
MAE: 10.864175563463819


# MLP

In [25]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=0.5, hidden_layer_sizes=(10))

In [26]:
clf.fit(x_train, y_train)

C:\Users\ozgur\anaconda3\envs\tf\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(alpha=0.5, hidden_layer_sizes=10, solver='lbfgs')

In [27]:
y_pred = clf.predict(x_test)

In [28]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
mse = mean_squared_error(y_test, y_pred)

# Calculate root mean squared error (RMSE)
rmse = np.sqrt(mse)

# Calculate R-squared (R2)
r2 = r2_score(y_test, y_pred)

# Calculate mean absolute error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Print the results
print("MSE:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)
print("MAE:", mae)

MSE: 161.22835112692763
RMSE: 12.697572647042726
R-squared: 0.3294850513908427
MAE: 7.23902728351127


# RNN

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from keras.layers import SimpleRNN

In [30]:
rnn = Sequential()

In [31]:
rnn.add(SimpleRNN(units = 45, return_sequences = True, input_shape = (x_train.shape[1], 1)))

In [32]:
rnn.add(Dropout(0.2))

In [33]:
rnn.add(SimpleRNN(units = 45, return_sequences = True))

rnn.add(Dropout(0.2))

rnn.add(SimpleRNN(units = 45, return_sequences = True))

rnn.add(Dropout(0.2))

rnn.add(SimpleRNN(units = 45))

rnn.add(Dropout(0.2))


In [34]:
rnn.add(Dense(units = 1))

In [35]:
rnn.compile(optimizer = 'adam', loss = 'mean_squared_error',metrics=["accuracy"])

In [36]:
rnn.fit(x_train, y_train, epochs = 100, batch_size = 32)


Epoch 1/100
107/107 [==============================] - 4s 13ms/step - loss: 381.5564 - accuracy: 0.0424
Epoch 2/100
107/107 [==============================] - 1s 13ms/step - loss: 282.4656 - accuracy: 0.0474
Epoch 3/100
107/107 [==============================] - 1s 13ms/step - loss: 262.7073 - accuracy: 0.0474
Epoch 4/100
107/107 [==============================] - 1s 13ms/step - loss: 244.5765 - accuracy: 0.0474
Epoch 5/100
107/107 [==============================] - 1s 13ms/step - loss: 220.6386 - accuracy: 0.0474
Epoch 6/100
107/107 [==============================] - 1s 12ms/step - loss: 205.6474 - accuracy: 0.0474
Epoch 7/100
107/107 [==============================] - 1s 13ms/step - loss: 200.4043 - accuracy: 0.0474
Epoch 8/100
107/107 [==============================] - 1s 14ms/step - loss: 192.1056 - accuracy: 0.0465
Epoch 9/100
107/107 [==============================] - 1s 13ms/step - loss: 187.4736 - accuracy: 0.0471
Epoch 10/100
107/107 [==============================] - 1s 13ms/

107/107 [==============================] - 1s 14ms/step - loss: 118.5824 - accuracy: 0.0453
Epoch 80/100
107/107 [==============================] - 2s 14ms/step - loss: 114.8782 - accuracy: 0.0444
Epoch 81/100
107/107 [==============================] - 2s 14ms/step - loss: 117.5032 - accuracy: 0.0438
Epoch 82/100
107/107 [==============================] - 2s 14ms/step - loss: 115.4828 - accuracy: 0.0430
Epoch 83/100
107/107 [==============================] - 1s 14ms/step - loss: 116.8716 - accuracy: 0.0427
Epoch 84/100
107/107 [==============================] - 1s 14ms/step - loss: 120.9640 - accuracy: 0.0462
Epoch 85/100
107/107 [==============================] - 2s 14ms/step - loss: 116.8730 - accuracy: 0.0433
Epoch 86/100
107/107 [==============================] - 2s 14ms/step - loss: 115.2502 - accuracy: 0.0444
Epoch 87/100
107/107 [==============================] - 1s 14ms/step - loss: 116.2829 - accuracy: 0.0433
Epoch 88/100
107/107 [==============================] - 1s 14ms/step

In [37]:
y_pred = rnn.predict(x_test)

53/53 [==============================] - 1s 5ms/step


In [38]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Assuming y_true and y_pred are arrays of actual and predicted values, respectively

# Calculate mean squared error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calculate root mean squared error (RMSE)
rmse = np.sqrt(mse)

# Calculate R-squared (R2)
r2 = r2_score(y_test, y_pred)

# Calculate mean absolute error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Print the results
print("MSE:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)
print("MAE:", mae)

MSE: 95.27299399906173
RMSE: 9.760788595142389
R-squared: 0.60377956960665
MAE: 6.595086325247234


In [39]:
clvp=pd.DataFrame(y_pred)
yv=pd.DataFrame(y_test)
display_side_by_side(yv.head(100),clvp.head(100), titles=['Actual','Predicted'])

,0
0,48
1,16
2,4
3,2
4,6
5,7
6,7
7,34
8,69
9,27
